In [2]:
PATH_PROCESSED = '../data/process/stack_overflow_processed.pkl'

DEVELOPER_TYPE = ['DevType']

WORKED_WITH_IN_PAST_YEARS = ['DatabaseWorkedWith', 'LanguageWorkedWith', 'MiscTechWorkedWith', 'PlatformWorkedWith', 'WebframeWorkedWith']
WANT_TO_WORK_WITH_NEXT_YEAR =  ['DatabaseDesireNextYear', 'LanguageDesireNextYear', 'MiscTechDesireNextYear', 'PlatformDesireNextYear'
                                , 'WebframeDesireNextYear']

## Load important packages

In [3]:
import pandas as pd
import numpy as np
import pickle

import matplotlib.pyplot as plt


from sklearn.preprocessing import StandardScaler, MultiLabelBinarizer, RobustScaler
from sklearn.cluster import AgglomerativeClustering
from sklearn.pipeline import make_pipeline
from sklearn.manifold import TSNE
from sklearn.preprocessing import OneHotEncoder

from scipy.cluster.hierarchy import dendrogram

In [4]:
# load Data
data = pd.read_pickle(PATH_PROCESSED)
data.head(3)

,Respondent,MainBranch,Hobbyist,Age,Age1stCode,CompFreq,CompTotal,ConvertedComp,Country,CurrencyDesc,...,SurveyEase,SurveyLength,Trans,UndergradMajor,WebframeDesireNextYear,WebframeWorkedWith,WelcomeChange,WorkWeekHrs,YearsCode,YearsCodePro
0,1,I am a developer by profession,Yes,NaN,13.0,Monthly,NaN,NaN,Germany,European Euro,...,Neither easy nor difficult,Appropriate in length,No,"Computer science, computer engineering, or sof...",[ASP.NET Core],"[ASP.NET, ASP.NET Core]",Just as welcome now as I felt last year,50.0,36.0,27.0
1,2,I am a developer by profession,No,NaN,19.0,NaN,NaN,NaN,United Kingdom,Pound sterling,...,NaN,NaN,NaN,"Computer science, computer engineering, or sof...",[],[],Somewhat more welcome now than last year,NaN,7.0,4.0
2,3,I code primarily as a hobby,Yes,NaN,15.0,NaN,NaN,NaN,Russian Federation,NaN,...,Neither easy nor difficult,Appropriate in length,NaN,NaN,[],[],Somewhat more welcome now than last year,NaN,4.0,NaN


### One Hot Enconing For the columns that we interested in

In [12]:
dataCopy = data.copy()
dataEncoded = {}

for column in DEVELOPER_TYPE + WORKED_WITH_IN_PAST_YEARS:
    binarizer = MultiLabelBinarizer()
    df = pd.DataFrame(binarizer.fit_transform(dataCopy[column]), columns=binarizer.classes_, index=dataCopy[column].index)
    dataEncoded[column] = df

In [13]:
# merge
dataCopy = pd.concat(dataEncoded, axis=1)
dataCopy

DevType                           \
      Academic researcher Data or business analyst   
0                       0                        0   
1                       0                        0   
2                       0                        0   
3                       0                        0   
4                       0                        0   
...                   ...                      ...   
64456                   0                        0   
64457                   0                        0   
64458                   0                        0   
64459                   0                        0   
64460                   0                        0   

                                                                            \
      Data scientist or machine learning specialist Database administrator   
0                                                 0                      0   
1                                                 0                      0   
2                                                 0                      0   
3                                                 0                      0   
4                                                 0                      0   
...                                             ...                    ...   
64456                                             0                      0   
64457                                             0                      0   
64458                                             0                      0   
64459                                             0                      0   
64460                                             0                      0   

                                                                            \
      Designer DevOps specialist Developer, QA or test Developer, back-end   
0            0                 0                     0                   0   
1            0                 0                     0                   0   
2            0                 0                     0                   0   
3            0                 0                     0                   0   
4            0                 0                     0                   0   
...        ...               ...                   ...                 ...   
64456        0                 0                     0                   0   
64457        0                 0                     0                   0   
64458        0                 0                     0                   0   
64459        0                 0                     0                   0   
64460        0                 0                     0                   0   

                                                     \
      Developer, desktop or enterprise applications   
0                                                 1   
1                                                 0   
2                                                 0   
3                                                 0   
4                                                 0   
...                                             ...   
64456                                             0   
64457                                             0   
64458                                             0   
64459                                             0   
64460                                             0   

                                                   ... WebframeWorkedWith  \
      Developer, embedded applications or devices  ...            Express   
0                                               0  ...                  0   
1                                               0  ...                  0   
2                                               0  ...                  0   
3                                               0  ...                  0   
4                                               0  ...                  0   
...                           

In [14]:
# calculate sum of all developer technologies
skills = dataCopy.drop(['DevType'], axis=1).sum().reset_index()
skills.columns = ['Group' ,'Technologies', 'Count']
skills

,Group,Technologies,Count
0,DatabaseWorkedWith,Cassandra,1654
1,DatabaseWorkedWith,Couchbase,937
2,DatabaseWorkedWith,DynamoDB,3497
3,DatabaseWorkedWith,Elasticsearch,6817
4,DatabaseWorkedWith,Firebase,7128
...,...,...,...
85,WebframeWorkedWith,Ruby on Rails,2944
86,WebframeWorkedWith,Spring,6941
87,WebframeWorkedWith,Symfony,1851
88,WebframeWorkedWith,Vue.js,7322


Visualize The number of tecnologthies based on groups using tree

In [17]:
import plotly.graph_objects as go
import plotly.express as px


fig = px.treemap(skills, path=['Group', 'Technologies'], values='Count',
                  color='Count', hover_data=['Technologies'],
                  color_continuous_scale='RdBu',
                  )
fig.update_layout(width=1400, height = 700)
fig.show()

C:\Users\TURBO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

C:\Users\TURBO\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\plotly\express\_core.py:1637: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



***Notice from the Tree Map that Most Used Language is JavaScript, Most platform Used Linux and Windows, Most Database is MySQL Most webframework is JQuery-React JS and finally most MiscTech is NodeJs*** 

## Creat Heatmap between jobs and skills

In [30]:
sorted_type = dataCopy['DevType'].sum().sort_values(ascending=False).index.to_list()
sorted_skills = dataCopy.drop(['DevType'], axis=1).sum().sort_values(ascending=False).droplevel(level=0).index.to_list()
